In [2]:
import sys
import mne
import scipy.io as sp
from scipy import interpolate
import numpy as np
import random
import pandas as pd
import multiprocessing as mp
import concurrent.futures
from mne.decoding import CSP
import pymrmr
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import logging
from scipy.io import loadmat
from scipy.signal import hamming
from scipy.signal import hann
from scipy.signal import blackman
from scipy.signal import kaiser
from scipy.signal import gaussian
from sklearn.decomposition import FastICA
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
# import lightgbm as lgb
# from catboost import CatBoostClassifier
# from sklearn.impute import KNNImputer
# from sklearn.decomposition import PCA
# from pyriemann.estimation import Covariances
# from pyriemann.tangentspace import TangentSpace
# from pyriemann.classification import MDM
from collections import Counter
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.signal import medfilt




In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)


# Set display options for NumPy
np.set_printoptions(threshold=np.inf)

# Initialization

In [4]:
WINDOW_TIME_LENGTH = 4
SAMPLING_RATE = 250
WINDOW_SAMPLE_LENGTH = WINDOW_TIME_LENGTH*SAMPLING_RATE
NUMBER_OF_CHANNELS = 64
beta = 1.5

epoch_length = 1000
sampling_freq = 250
number_of_runs = 10
number_of_components = 10
number_of_selected_features = 10
number_of_processes = 10
number_of_bands = 9
column_names = ['participant', 'class1', 'class2','running_time','test_acc','train_acc','test_size','train_size','train_block','test_block']
column_names_v2 = ['participant', 'class1', 'class2','running_time','test_acc','train_acc','test_size','train_size','train_block','test_block','test_acc_vote']


trial_order=[['Tongue','Feet','Mis','Hand'],
            ['Feet','Mis','Hand','Tongue'],
            ['Hand','Feet','Tongue','Mis'],
            ['Tongue','Mis','Hand','Feet'],
            ['Mis','Feet','Hand','Tongue'],
            ['Feet','Hand','Tongue','Mis'],
            ['Hand','Tongue','Mis','Feet'],
            ['Tongue','Feet','Mis','Hand'],
            ['Mis','Tongue','Hand','Feet']]


# Functions

In [5]:
def get_task_rest_times(b_num):
    if b_num == 0:
        task_time = [[12, 16, 20, 8],
                    [16, 12, 20, 8],
                    [20, 16, 8, 12],
                    [20, 12, 8, 16]]
        
        rest_time = [[20, 8, 16, 12],
                    [16, 20, 8, 12],
                    [12, 20, 16, 8],
                    [20, 12, 8, 16]]
        
    elif b_num == 1:
        task_time = [[12, 8, 20, 16],
                    [16, 20, 8, 12],
                    [8, 20, 16, 12],
                    [8, 12, 20, 16]]
        
        rest_time = [[16, 12, 8, 20],
                    [8, 20, 12, 16],
                    [20, 16, 8, 12],
                    [12, 16, 20, 8]]
        
    elif b_num == 2:
        task_time = [[16, 8, 12, 20],
                    [20, 16, 12, 8],
                    [12, 20, 8, 16],
                    [8, 12, 16, 20]]
        
        rest_time = [[8, 20, 16, 12],
                    [12, 8, 20, 16],
                    [16, 12, 20, 8],
                    [8, 12, 20, 16]]
        
    elif b_num == 3:
        task_time = [[12, 16, 20, 8],
                    [16, 12, 20, 8],
                    [20, 16, 8, 12],
                    [20, 12, 8, 16]]
        
        rest_time = [[20, 8, 16, 12],
                    [16, 20, 8, 12],
                    [12, 20, 16, 8],
                    [20, 12, 8, 16]]
        
    elif b_num == 4:
        task_time = [[16, 8, 20, 12],
                    [12, 16, 8, 20],
                    [20, 8, 12, 16],
                    [8, 20, 12, 16]]
        
        rest_time = [[8, 12, 16, 20],
                    [16, 20, 12, 8],
                    [12, 16, 8, 20],
                    [20, 8, 12, 16]]
        
    elif b_num == 5:
        task_time = [[16, 12, 8, 20],
                    [20, 16, 12, 8],
                    [8, 16, 20, 12],
                    [12, 8, 16, 20]]

        rest_time = [[12, 8, 16, 20],
                    [16, 8, 20, 12],
                    [20, 12, 16, 8],
                    [8, 16, 12, 20]]
        
    elif b_num == 6:
        task_time = [[16, 8, 12, 20],
                    [20, 8, 16, 12],
                    [8, 16, 12, 20],
                    [16, 20, 12, 8]]

        rest_time = [[16, 8, 12, 20],
                    [12, 20, 8, 16],
                    [20, 16, 12, 8],
                    [8, 16, 20, 12]]     
    elif b_num ==7:
        task_time = [[12, 8, 20, 16],
                    [16, 20, 8, 12],
                    [8, 20, 16, 12],
                    [8, 12, 20, 16]]   
               
        rest_time = [[16, 12, 8, 20],
                    [8, 20, 12, 16],
                    [20, 16, 8, 12],
                    [12, 16, 20, 8]]  
    
    elif b_num == 8:
        task_time = [[16, 8, 12, 20],
                    [20, 16, 12, 8],
                    [12, 20, 8, 16],
                    [8, 12, 16, 20]]
        
        rest_time = [[8, 20, 16, 12],
                    [12, 8, 20, 16],
                    [16, 12, 20, 8],
                    [8, 12, 20, 16]]
        
    else:
        raise("Error in block number")
    

    return task_time,rest_time


In [6]:
def trial_times_genertor(task_times,rest_times):
    block_times = [item for pair in zip(task_times, rest_times) for item in pair]
    return block_times
    

In [7]:

def fill_zeros_with_average(matrix):
    # Iterate through the matrix
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            for k in range(matrix.shape[2]):
                if matrix[i, j, k] == 0:
                    # Find the neighboring non-zero elements
                    neighbors = []
                    if i > 0 and matrix[i - 1, j, k] != 0:
                        neighbors.append(matrix[i - 1, j, k])
                    if i < matrix.shape[0] - 1 and matrix[i + 1, j, k] != 0:
                        neighbors.append(matrix[i + 1, j, k])
                    if j > 0 and matrix[i, j - 1, k] != 0:
                        neighbors.append(matrix[i, j - 1, k])
                    if j < matrix.shape[1] - 1 and matrix[i, j + 1, k] != 0:
                        neighbors.append(matrix[i, j + 1, k])
                    if k > 0 and matrix[i, j, k - 1] != 0:
                        neighbors.append(matrix[i, j, k - 1])
                    if k < matrix.shape[2] - 1 and matrix[i, j, k + 1] != 0:
                        neighbors.append(matrix[i, j, k + 1])

                    # Fill the zero with the average of neighboring non-zero values
                    if neighbors:
                        matrix[i, j, k] = sum(neighbors) / len(neighbors)

    return matrix

In [8]:
def calc_csp(x_train, y_train, x_test,number_of_components):
    # csp = CSP(n_components=number_of_components, reg='ledoit_wolf', log=True)
    csp = CSP(number_of_components)



    
    # x_train = fill_zeros_with_average(x_train)
    # x_train = np.add(x_train, 0.000001)



    # nan_count = np.isnan(x_train).sum()
    # print("Number of NaN values:", nan_count)

    # empty_field_count = np.count_nonzero(x_train == 0)
    # print("Number of empty fields:", empty_field_count)

    # zeros_locations_3d = np.where(x_train == 0)
    # print("Locations of zeros:", zeros_locations)
    
# Printing indices and corresponding values
    # for depth_idx, row_idx, col_idx in zip(zeros_locations_3d[0], zeros_locations_3d[1], zeros_locations_3d[2]):
    #     value_at_zero_location = x_train[depth_idx, row_idx, col_idx]
    #     print(f"Zero found at position ({depth_idx}, {row_idx}, {col_idx}) with value {value_at_zero_location}")


    csp_fit = csp.fit(x_train, y_train)
    train_feat = csp_fit.transform(x_train)
    test_feat = csp_fit.transform(x_test)
    return train_feat, test_feat

In [9]:
def class_extractor(number_of_epochs, class_1, class_2, data, labels):
    size = sum(labels[:,0] == class_1) + sum(labels[:,0] == class_2)
    Final_labels = np.zeros((size,1)).astype(int)
    dataset = np.zeros((size,num_channels, epoch_length))
    index = 0
    for i in range(number_of_epochs):
        if labels[i,0] == class_1 or labels[i,0] == class_2:
            dataset[index,:,:] = data[i,:,:]
            Final_labels[index,0] = labels[i,0]
            index = index + 1
        else:
            continue
            
    return dataset, Final_labels

In [10]:
def feature_extractor(dataset, labels, number_of_bands, test_data,number_of_components):

    low_cutoff = 0
    
    for b in range(number_of_bands):
        logging.getLogger('mne').setLevel(logging.WARNING)
        low_cutoff += 4
        data = dataset.copy()
        data_test = test_data.copy() 
        print("Frequency range: ",low_cutoff)
        filtered_data = mne.filter.filter_data(data, sampling_freq, low_cutoff, low_cutoff + 4, verbose = False, n_jobs = 4)
        filtered_data_test = mne.filter.filter_data(test_data, sampling_freq, low_cutoff, low_cutoff + 4, verbose = False, n_jobs = 4)

        #PCA
        # from mne.decoding import UnsupervisedSpatialFilter
        # from sklearn.decomposition import PCA, FastICA

        # pca = UnsupervisedSpatialFilter(PCA(64), average=False)
        # pca_fit = pca.fit(filtered_data)
        # filtered_data = pca_fit.transform(filtered_data)
        # filtered_data_test = pca_fit.transform(filtered_data_test)
        # train_feats = filtered_data
        # test_feats = filtered_data_test

        # filtered_data = data
        # filtered_data_test = data_test
        
        [train_feats, test_feats] = calc_csp(filtered_data, labels[:,0], filtered_data_test,number_of_components)
        if b == 0:
            train_features = train_feats
            test_features = test_feats
        else:
            train_features = np.concatenate((train_features, train_feats), axis = 1)
            test_features = np.concatenate((test_features, test_feats), axis = 1)
    
    return train_features, test_features

In [11]:
def feature_selector(train_features, labels, number_of_selected_features):
    X = pd.DataFrame(train_features)
    y = pd.DataFrame(labels)
    K = number_of_selected_features
    
    df = pd.concat([y,X], axis = 1)
    df.columns = df.columns.astype(str)
        
    selected_features = list(map(int, pymrmr.mRMR(df, 'MID', K)))
    return selected_features

In [12]:
def data_reader(path,p_num,block_list):
    data_dict = {}
    for b_num in block_list:
        print(b_num)
        mat = loadmat(path+'P'+str(p_num)+'B'+str(b_num)+'.mat', chars_as_strings=True, mat_dtype=True, squeeze_me=True, struct_as_record=False, verify_compressed_data_integrity=False, variable_names=None)
        df = pd.DataFrame(mat['Data'])
        data_dict[b_num] = df
    return data_dict


In [13]:
def get_group_start_indices(dataframe):
    group_indices = []
    current_label = None

    for idx, row in dataframe.iterrows():
        if row.iloc[-1] != current_label:
            group_indices.append(idx)
            current_label = row.iloc[-1]

    return group_indices

In [14]:
def extra_samples_counter(df,class_1,class_2):
    x=0
    i=0
    sampleList = []
    while i<len(df):
        if (df.iloc[i,-1]==class_1):
            x+=1
        else:
            i-=1
            sampleList.append(x)
            x=0
            class_1,class_2 = class_2,class_1
        i+=1
    sampleList.append(x)
    print(sampleList)
    

In [15]:
def extra_samples_block_counter(df,trial_order,b_num):

    df.drop(df[df.iloc[:,-1].isin(['Begin', 'End'])].index, inplace=True)
    df.reset_index(drop=True, inplace=True)    
    df['group'] = (df.iloc[:,-1] != df.iloc[:,-1].shift(1)).cumsum()

    
    group_counts_Rest = df[df.iloc[:,-1] == 'Rest'].groupby('group').size()
    with open('sampleList.txt', 'a') as file:
        file.write(f'block {b_num+1} '+'\n')
        for j in range (len(trial_order)):
            print(trial_order[j])
            trial_num = j
            task_times,rest_times = get_task_rest_times(b_num)
            trial_times = trial_times_genertor(task_times[trial_num],rest_times[trial_num])
            trial_samples = [item*SAMPLING_RATE for item in trial_times]
            group_counts_task = df[df.iloc[:,-1] == trial_order[j]].groupby('group').size()
            sampleList = []
            for i in range(4):
                task = group_counts_task.iloc[i]
                rest = group_counts_Rest.iloc[4*j+i]
                sampleList.append(task)
                sampleList.append(rest)
            # extra_samples = [x-y for x,y in zip(sampleList,trial_samples)]
            file.write(', '.join(map(str, sampleList)) + f' trial={trial_order[j]} '+'\n')
            print(sampleList)
        file.write('\n\n')


In [16]:
def data_cleaner(df,class_1,class_2,tasks_time):
    # sys.exit() 
    class_x = class_1
    class_y = class_2
    new_df = pd.DataFrame()
    trial_df = df.copy() 
    # print(tasks_time)
    for i in range(len(tasks_time)):
        sample_point = tasks_time[i]*SAMPLING_RATE
        if(trial_df.iloc[sample_point+1,-1] == class_x ):
            if(i==len(tasks_time)-1):
                temp_df = trial_df.iloc[:sample_point,:]
                new_df = pd.concat([new_df, temp_df], axis=0)
                new_df.reset_index(drop=True, inplace=True)
            else:    
                temp_df = trial_df.iloc[:sample_point,:]
                next_task_idx = trial_df[trial_df.iloc[:, -1] == class_y].index
                trial_df.drop(trial_df.index[0:next_task_idx[0]], inplace=True)
                trial_df.reset_index(drop=True, inplace=True)
                new_df = pd.concat([new_df, temp_df], axis=0)
                new_df.reset_index(drop=True, inplace=True)
                class_x,class_y = class_y,class_x

    return new_df

In [17]:
def class_seperator(cleaned_df,class_1,class_2):
    df = cleaned_df
    sorting_order = {class_1: 0, class_2: 1}

    df['sorting_order'] = df.iloc[:, -1].map(sorting_order)
    df.sort_values(by=['sorting_order', df.columns[-1]], inplace=True)
    df.drop('sorting_order', axis=1, inplace=True)
    df.reset_index(drop=True, inplace=True)

    return df

In [18]:
def shuffler(dataset,labels):
    print(dataset.shape)
    print(labels.shape)
    np.random.seed(42)
    indices = np.random.permutation(len(dataset))
    shuffled_dataset = dataset[indices]
    shuffled_labels = labels[indices]
    return shuffled_dataset,shuffled_labels
    

In [19]:
def cal_epoch(df_len,sliding_len,window_len):
    # print(window_len,sliding_len,df_len)
    number_of_epochs = int((int(df_len-window_len)/sliding_len)) +1
    return number_of_epochs

In [20]:
def data_label_attacher(cleaned_df,class_1,class_2,random_flag,class_seperator_flag,sliding_time,window_time_length,window_type,number_of_channels):
    SLIDING_POINTS = int(sliding_time*SAMPLING_RATE)
    window_time = window_time_length
    new_df_ = cleaned_df.copy()
    new_df_.drop(cleaned_df.columns[-1], axis=1, inplace=True)
    X = new_df_.to_numpy()
    X = np.transpose(X)
    number_of_epochs = cal_epoch(int(int(len(cleaned_df)/SAMPLING_RATE)),sliding_time,window_time)
    # print(number_of_epochs)
    dataset = np.zeros((number_of_epochs,number_of_channels,WINDOW_SAMPLE_LENGTH))
    labels = np.zeros((number_of_epochs,1)).astype(int)

    index = get_group_start_indices(cleaned_df)
    index.append(len(cleaned_df))
    k = 0  
    startIdx = int(k * WINDOW_SAMPLE_LENGTH)
    endIdx = int((k+1) * WINDOW_SAMPLE_LENGTH )
    l = 0
    label = 1
    for i in range(number_of_epochs):
        
        if(startIdx>=index[l] and endIdx<=index[l+1]):
            # print(startIdx,endIdx,index[l],index[l+1],"start, end, index[l], index[l+1] in if")
            slice_X = X[:, startIdx:endIdx]

            if window_type == "Kaiser":
                kaiser_window = kaiser(WINDOW_SAMPLE_LENGTH,beta)
                slice_X *= kaiser_window

            elif window_type == "Hamming":
                hamming_window = hamming(WINDOW_SAMPLE_LENGTH)
                slice_X *= hamming_window
            
            elif window_type == "Hanning":
                hanning_window = hann(WINDOW_SAMPLE_LENGTH)
                slice_X *= hanning_window
            
            elif window_type == "Rec":
                pass

            else:
                raise ValueError("Window type is wrong!")

            dataset[i, :, :] = slice_X
            labels[i,0] = label
            # print("i is: ",i)
            # print("label is: ",label)

        else:
            
            temp = endIdx-index[l+1]
            # print(temp,endIdx,index[l+1],"temp,end,index l+1")
            slice_X = X[:, startIdx:endIdx]
            if window_type == "Kaiser":
                kaiser_window = kaiser(WINDOW_SAMPLE_LENGTH,beta)
                slice_X *= kaiser_window

            elif window_type == "Hamming":
                hamming_window = hamming(WINDOW_SAMPLE_LENGTH)
                slice_X *= hamming_window
            
            elif window_type == "Hanning":
                hanning_window = hann(WINDOW_SAMPLE_LENGTH)
                slice_X *= hanning_window
            
            elif window_type == "Rec":
                pass
            
            else:
                raise ValueError("Window type is wrong!")
            dataset[i, :, :] = slice_X

            if(temp<WINDOW_SAMPLE_LENGTH/2):
                # print("i is: ",i)
                # print("label is: ",label)
                labels[i,0] = label
            else:
                labels[i,0] = int(not(label))
                # print("i is: ",i)
                # print("label is: ",int(not(label)))

            if(startIdx>=index[l+1]):
                l+=1
                # print(f"label changed in i = {i}")
                label = int(not(label))

                

            

        startIdx+=SLIDING_POINTS
        endIdx+=SLIDING_POINTS
    








        # a = df_len - wdinow_len
        # a/sliding_len
        # b = a%sliding_len





####################################################


    # new_df_ = cleaned_df.copy()
    # new_df_.drop(cleaned_df.columns[-1], axis=1, inplace=True)
    # X = new_df_.to_numpy()
    # X = np.transpose(X)
    # number_of_epochs = int(len(new_df_)/WINDOW_SAMPLE_LENGTH)
    # number_of_epochs = int((int(len(new_df_))-WINDOW_SAMPLE_LENGTH)/SLIDING_POINTS) +1

    
    # dataset = np.zeros((number_of_epochs,NUMBER_OF_CHANNELS,WINDOW_SAMPLE_LENGTH))
    # labels = np.zeros((number_of_epochs,1)).astype(int)
















    
    # #Initialization
    # if class_seperator_flag:
    #     seperated_class_df = class_seperator(cleaned_df,class_1,class_2)
    #     new_df_ = seperated_class_df.copy()
    #     new_df_.drop(seperated_class_df.columns[-1], axis=1, inplace=True)
    #     X = new_df_.to_numpy()
    #     X = np.transpose(X)
    #     empty_field_count = np.count_nonzero(X == 0)
    #     print("Number of empty fields in X:", empty_field_count)
    #     # zero_indices = np.where(X == 0)
    #     # print("befor filling",len(zero_indices[0]))
    #     # X[zero_indices] += 0.001
    #     # zero_indices = np.where(X == 0)
    #     # print("after filling",len(zero_indices[0]))
    #     number_of_epochs = int((int(len(new_df_))-WINDOW_SAMPLE_LENGTH)/TR_SLIDING_POINTS)
    #     print(number_of_epochs)
    # else :  
    #     new_df_ = cleaned_df.copy()
    #     new_df_.drop(cleaned_df.columns[-1], axis=1, inplace=True)
    #     X = new_df_.to_numpy()
    #     X = np.transpose(X)
    #     empty_field_count = np.count_nonzero(X == 0)
    #     print("Number of empty fields in X:", empty_field_count)
    #     # zero_indices = np.where(X == 0)
    #     # print("befor filling",len(zero_indices[0]))
    #     # X[zero_indices] += 0.001
    #     # zero_indices = np.where(X == 0)
    #     # print("after filling",len(zero_indices[0]))

    #     number_of_epochs = int(len(new_df_)/WINDOW_SAMPLE_LENGTH)

    # dataset = np.zeros((number_of_epochs,NUMBER_OF_CHANNELS,WINDOW_SAMPLE_LENGTH))
    # labels = np.zeros((number_of_epochs,1)).astype(int)

    # if class_seperator_flag:
    #     i = 0  
    #     startIdx = i * WINDOW_SAMPLE_LENGTH
    #     endIdx = (i+1) * WINDOW_SAMPLE_LENGTH 
    #     while(endIdx<=int(len(new_df_))/2):
    #         slice_X = X[:, startIdx:endIdx]

    #         kaiser_window = kaiser(WINDOW_SAMPLE_LENGTH,beta)
    #         slice_X *= kaiser_window

    #         dataset[i, :, :] = slice_X
    #         labels[i,0] = 0
    #         # if (seperated_class_df.iloc[startIdx, 64] == class_1):
    #         #     labels[i,0] = 0
    #         # elif(seperated_class_df.iloc[startIdx, 64] == class_2):
    #         #     labels[i,0] = 1
    #         # else:
    #         #     labels[i,0] = 2
    #         startIdx+=TR_SLIDING_POINTS
    #         endIdx+=TR_SLIDING_POINTS
    #         i+=1
    #     # print(int(len(new_df_))/2,"len")    
    #     # print(endIdx,"endIdx")    
    #     # print(seperated_class_df.iloc[endIdx-2:endIdx+2,64])
       
    #     j = i
        
    #     startIdx = endIdx-TR_SLIDING_POINTS
    #     endIdx = startIdx+WINDOW_SAMPLE_LENGTH
    #     print(j, "j is this")
    #     while(endIdx<=int(len(new_df_))):
    #         slice_X = X[:, startIdx:endIdx]

    #         kaiser_window = kaiser(WINDOW_SAMPLE_LENGTH,beta)
    #         slice_X *= kaiser_window

    #         dataset[j, :, :] = slice_X
    #         labels[j,0] = 1
    #         # if (cleaned_df.iloc[startIdx, 64] == class_1):
    #         #     labels[j,0] = 0
    #         # elif(cleaned_df.iloc[startIdx, 64] == class_2):
    #         #     labels[j,0] = 1
    #         # else:
    #         #     labels[j,0] = 2
    #         startIdx+=TR_SLIDING_POINTS
    #         endIdx+=TR_SLIDING_POINTS
    #         j+=1
    #     print(j, "j is this")
    #     # dataset,labels = shuffler(dataset,labels)

    # else:
    #     i = 0  
    #     start_idx = i * WINDOW_SAMPLE_LENGTH
    #     end_idx = (i+1) * WINDOW_SAMPLE_LENGTH 
    #     while (end_idx<=int(len(new_df_))):
    #         slice_X = X[:, start_idx:end_idx]

    #         kaiser_window = kaiser(WINDOW_SAMPLE_LENGTH,beta)
    #         slice_X *= kaiser_window
            
    #         dataset[i, :, :] = slice_X
    #         if (cleaned_df.iloc[start_idx, 64] == class_1):
    #             labels[i,0] = 0
    #         elif(cleaned_df.iloc[start_idx, 64] == class_2):
    #             labels[i,0] = 1
    #         else:
    #             labels[i,0] = 2
    #         start_idx+=SLIDING_POINTS
    #         end_idx+=SLIDING_POINTS
    #         i+=1
    #     # dataset,labels = shuffler(dataset,labels)



#####################################################













    #For training and test purpose
    # if random_flag:
    #     randomlist = random.sample(range(number_of_epochs), number_of_epochs)
    # else:
    #     randomlist = list(range(number_of_epochs))
    #Labeling the data



    # for i in range(number_of_epochs):
    #     start_idx = randomlist[i] * WINDOW_SAMPLE_LENGTH + SLIDING_POINTS
    #     end_idx = (randomlist[i] + 1) * WINDOW_SAMPLE_LENGTH
    #     slice_X = X[:, start_idx:end_idx]

    #     # hamming_window = hamming(WINDOW_SAMPLE_LENGTH)
    #     # slice_X *= hamming_window

    #     # hanning_window = hann(WINDOW_SAMPLE_LENGTH)
    #     # slice_X *= hanning_window

    #     # blackman_window = blackman(WINDOW_SAMPLE_LENGTH)
    #     # slice_X *= blackman_window

    #     # kaiser_window = kaiser(WINDOW_SAMPLE_LENGTH,0.5)
    #     # slice_X *= kaiser_window

    #     # gaussian_window = gaussian(WINDOW_SAMPLE_LENGTH,0.5)
    #     # slice_X *= gaussian_window


    #     dataset[i, :, :] = slice_X
    #     if (cleaned_df.iloc[randomlist[i] * WINDOW_SAMPLE_LENGTH, 64] == class_1):
    #         labels[i,0] = 0
    #     elif(cleaned_df.iloc[randomlist[i] * WINDOW_SAMPLE_LENGTH, 64] == class_2):
    #         labels[i,0] = 1
    #     else:
    #         labels[i,0] = 2
    
    # empty_field_count = np.count_nonzero(dataset == 0)
    # print("Number of empty fields in dataset:", empty_field_count,"dataset shape",dataset.shape)
    # print(labels)
    return dataset,labels




In [21]:
def trial_cutter(data, class_1):
    df = data.copy()
    Begin_trigger = "Begin" + "_" + class_1
    End_trigger = "End" + "_" + class_1
    Begin_idx = df[df.iloc[:, -1] == Begin_trigger].index
    End_idx = df[df.iloc[:, -1] == End_trigger].index
    trial_df = df.iloc[Begin_idx[0]+1:End_idx[0],:]
    trial_df.reset_index(drop=True, inplace=True)
    trial_df.head()
    return trial_df

In [22]:
def Begin_End_trigger_modifier(data):
    df = data.copy()
    Begin_indexes = df[df.iloc[:, -1] == 'Begin'].index
    End_indexes = df[df.iloc[:, -1] == 'End'].index
    if(len(Begin_indexes)==len(End_indexes)):
        for i in range(len(Begin_indexes)):
            index = Begin_indexes[i]+1
            val = df.iloc[index,-1]
            df.iloc[Begin_indexes[i],-1] = "Begin" + "_" + str(val)
            df.iloc[End_indexes[i],-1]   =  "End" + "_" + str(val)
    else:
        raise ValueError("Trigger seinding Exception")
    
    return df

In [23]:
def preprocessor(data_,class_1,class_2,tasks_time,set_type,clean_flag,sliding_time,window_time_length,window_type,number_of_channels):
    CLASS_1 = class_1
    CLASS_2 = class_2
    df = data_.copy()
    modified_df = Begin_End_trigger_modifier(df)
    trial_df = trial_cutter(modified_df,CLASS_1)
    # print(trial_df.shape,"trial_df")
    indexes = get_group_start_indices(trial_df)
    # print(indexes,'tasks index starting point')
    if clean_flag:
        cleaned_df = data_cleaner(trial_df,CLASS_1,CLASS_2,tasks_time)
        final_df = cleaned_df.copy()
    else:
        final_df = trial_df.copy()
    # print(final_df.shape,"final_df")

    if set_type =="TRAIN":
        random_flag = True
    elif set_type =="TEST":
        random_flag = False
    else:
        raise("Error in set type")

  
    final_data, final_labels = data_label_attacher(final_df,CLASS_1,CLASS_2,random_flag,clean_flag,sliding_time,window_time_length,window_type,number_of_channels)
      
    # print(final_data.shape,"final_data shape")
    # print(final_labels.shape,"final_labels shape")
    
    return final_data,final_labels

In [24]:
def trials_set_builder(data_dict,blocks_set,set_label,class_1,class_2,clean_flag,sliding_time,window_time_length,window_type,channels_to_remove,number_of_channels):
                                   
    counter = 0

    for b_num in blocks_set:
        trial_num = trial_order[b_num].index(class_1)
        task_times,rest_times = get_task_rest_times(b_num)
        # print(task_times[trial_num],rest_times[trial_num])
        trial_times = trial_times_genertor(task_times[trial_num],rest_times[trial_num])
        # print(trial_times)
        # print(data_dict[b_num].shape,"shape befor removal")

        # data = remove_outliers_across_channels(data_dict[b_num],10)
        # data = remove_outliers(data_dict[b_num])
        data = data_dict[b_num]
        # data = apply_median_filter(data,9)
        if class_1== 'Tongue' or class_1 == 'Mis':
            data = channel_remover(data,channels_to_remove)
            number_of_channels =  NUMBER_OF_CHANNELS-len(channels_to_remove)
        else:
            number_of_channels = NUMBER_OF_CHANNELS
        # print(number_of_channels,"number_of_channels")
        # print(data.shape,"shape after removal")
        # sys.exit()
        df = data.copy()
        # last_column = df.pop(df.columns[-1])
        # df.drop(df.columns[-1], axis=1, inplace=True)
        # eeg_data = df.to_numpy().T  # Transpose to have channels in columns

        # channel_names = [f'Ch{i+1}' for i in range(63)]

        # # Create MNE-Python RawArray object
        # info = mne.create_info(ch_names=channel_names, sfreq=sampling_freq, ch_types='eeg')
        # raw = mne.io.RawArray(eeg_data, info)

        # # Apply ICA
        # ica = mne.preprocessing.ICA(n_components=20, random_state=97, max_iter=800)
        # ica.fit(raw)
        # ica_components = ica.get_components()

        # # Convert the ICA components to a DataFrame
        # df2 = pd.DataFrame(data=ica_components.T, columns=channel_names)
        # df2 = df2.assign(LastColumn=last_column)
        # # df = data.copy(deep=False)
        dataset,labels = preprocessor(df,class_1,class_2,trial_times,set_label,clean_flag,sliding_time,window_time_length,window_type,number_of_channels)
        # print(dataset.shape)

        if counter == 0 :
            final_data = dataset
            final_labels = labels
            # print("Before concatenation - final_data shape:", final_data.shape, "dataset shape:", dataset.shape)
        else:
            final_data = np.vstack((final_data, dataset))
            final_labels = np.vstack((final_labels, labels))
            print("After concatenation - final_data shape:", final_data.shape, "final_labels shape:", final_labels.shape)

        counter+=1 
    # empty_field_count = np.count_nonzero(final_data == 0)
    # print("Number of empty fields in final_data:", empty_field_count,"final_data shape",final_data.shape)
    return final_data,final_labels

In [25]:
def find_duplicates(data_list):
    counted_values = Counter(data_list)
    duplicate_values = {value: count for value, count in counted_values.items() if count > 1}
    return duplicate_values

In [26]:
def Statistical_analysor(p_num_list,data_dicts_list):

    with open('/home/mahdi146/projects/def-b09sdp/mahdi146/Cedar/Classification/EEG/Classification/Statistics.txt', 'w') as file:
        for p in range(len(p_num_list)):
            file.write(f'Particpant: {p+3} '+'\n')
            for b in range(7):
                file.write(f'Block: {b+1} '+'\n')
                data_pd = data_dicts_list[p][b]
                data = data_pd.iloc[:, :-1]
                data_np = data.values
                eeg_data = data_np
                print("Data type:", type(eeg_data))
                print("Shape:", eeg_data.shape)
                eeg_data = np.array(eeg_data)
                mean_values = np.mean(eeg_data, axis=0)
                variance_values = np.var(eeg_data, axis=0)
                std_deviation_values = []
                
                for i in range(num_channels):
                    print(f"Channel {i + 1}:")
                    print(f"Mean: {mean_values[i]}")
                    print(f"Variance: {variance_values[i]}")
                    std_deviation_values.append(np.sqrt(variance_values[i]))
                    print(f"Standard Deviation: {std_deviation_values[i]}")
                    print()
                    file.write(f'Channel {i+1}: '+'\n')
                    file.write(f"Mean: {mean_values[i]}"+"\n")
                    file.write(f"Variance: {variance_values[i]}"+"\n")
                    file.write(f"Standard Deviation: {std_deviation_values[i]}"+"\n\n")
                
                lists_to_check = {
                'mean_values': mean_values,
                'variance_values': variance_values,
                'std_deviation_values': std_deviation_values
                }
                for list_name, data_list in lists_to_check.items():
                    duplicate_values = find_duplicates(data_list)
                    if duplicate_values:
                        print(f"Duplicate values and their counts for {list_name}:")
                        file.write(f"Duplicate values and their counts for {list_name}:"+"\n")
                        for value, count in duplicate_values.items():
                            print(f"Value: {value}, Count: {count}")
                            file.write(f"Value: {value}, Count: {count}"+"\n")
                    else:
                        print(f"No duplicate values found in the {list_name} list.")
                        file.write(f"No duplicate values found in the {list_name} list."+"\n")


In [27]:
def custom_accuracy(y_true, y_pred):
    mismatches = []
    total = len(y_true)
    mismatch_count = 0
    
    for i, (true_label, pred_label) in enumerate(zip(y_true, y_pred)):
        if true_label != pred_label:
            mismatches.append(i)
            mismatch_count += 1
            
    accuracy = 1 - (mismatch_count / total)
    
    return accuracy, mismatch_count, mismatches

In [28]:
def majority_vote_sliding_with_next(prediction_list, window_size=3):
    majority_votes = []
    
    for i in range(len(prediction_list) - window_size + 1):
        window = prediction_list[i:i+window_size]
        window_tuple = tuple(window)
        counts = Counter(window_tuple)
        majority = counts.most_common(1)[0][0]
        majority_votes.append(majority)
        
    return majority_votes


In [29]:
def majority_vote_sliding_with_prev(prediction_list, window_size=3):
    majority_votes = []
    
    for i in range(len(prediction_list)):
        if i >= window_size - 1:
            start_index = i - window_size + 1
            window = prediction_list[start_index:i+1]
            counts = Counter(window)
            majority = counts.most_common(1)[0][0]
            majority_votes.append(majority)
        
    return majority_votes

In [30]:
def majority_vote_sliding_with_prev_v2(prediction_list, window_size=3):
    majority_votes = []
    
    for i in range(len(prediction_list)):
        start_index = max(0, i - window_size + 1)
        window = prediction_list[start_index:i+1]
        counts = Counter(window)
        majority = counts.most_common(1)[0][0]
        majority_votes.append(majority)
        
    return majority_votes

In [31]:
def channel_remover(df, channels):
    
    df_copy = df.copy()
    df_copy.drop(df.columns[channels], axis=1, inplace=True)
    return df_copy

# Reading Data

In [32]:
block_list = [0,1,2,3,4,5,6]
p_num_list = [3,4,5,6,7,9,10,11,13,14]
data_dicts_list = []
for p_num in p_num_list:
    print(f'reading P{p_num}')
    data_dict = data_reader(f'../../Participants/P{p_num}/', p_num, block_list)
    data_dicts_list.append(data_dict)

reading P3
0


FileNotFoundError: [Errno 2] No such file or directory: '../../Participants/P3/P3B0.mat'

In [34]:
block_list = [0,1,2,3,4,5,6]
p_num_list = [3,4,5,6,7,9,10,11,13,14]
data_dicts_list = []
for p_num in p_num_list:
    print(f'reading P{p_num}')
    data_dict = data_reader(f'/home/mahdi146/projects/def-b09sdp/mahdi146/Cedar/Classification/Participants/P{p_num}/',p_num,block_list)
    data_dicts_list.append(data_dict)


reading P9
0


In [43]:
print(data_dicts_list[0][6].shape)

(114272, 65)


In [71]:
def apply_pca(train_data, test_data=None, n_components=50):
    # Reshape train data
    train_data_reshaped = np.reshape(train_data, (train_data.shape[0], -1))


    scaler = StandardScaler()
    scaled_train_data = scaler.fit_transform(train_data_reshaped)
    # scaled_train_data = train_data_reshaped
    
    # Apply PCA on train data
    pca = PCA(n_components=n_components)
    pca.fit(scaled_train_data)
    transformed_train_data = pca.transform(scaled_train_data)

    explained_variance_ratio = pca.explained_variance_ratio_
    cumulative_explained_variance = np.cumsum(explained_variance_ratio)
    print("Explained Variance Ratio:")
    print(explained_variance_ratio)
    print("\nCumulative Explained Variance:")
    print(cumulative_explained_variance)


    test_data_reshaped = np.reshape(test_data, (test_data.shape[0], -1))
    scaled_test_data = scaler.transform(test_data_reshaped)
    transformed_test_data = pca.transform(scaled_test_data)

    return transformed_train_data, transformed_test_data


In [ ]:
# def runs_iterattor(runs,clf,train_features,Y_tr,test_features,Y_te,test_acc_list,train_acc_list,vote_acc_list):

#     for r in range(runs):
#         clf.fit(train_features, Y_tr)
        
#         y_pr_te = clf.predict(test_features)
#         y_pr_tr = clf.predict(train_features)

#         accuracy_te = accuracy_score(Y_te, y_pr_te)
#         test_acc_list.append(accuracy_te)

#         accuracy_tr = accuracy_score(Y_tr,y_pr_tr)
#         train_acc_list.append(accuracy_tr)

#         y_pr_te_Vote = majority_vote_sliding_with_prev_v2(y_pr_te,vote_window)
#         Y_te_Vote = majority_vote_sliding_with_prev_v2(Y_te.reshape(-1),vote_window)
#         vote_acc, num_of_mismatches ,mismatches_list = custom_accuracy(Y_te_Vote,y_pr_te_Vote)
#         vote_acc_list.append(vote_acc)
#     return train_acc_list,test_acc_list,vote_acc_list



In [ ]:
# def class_iterator(class_1_list,data_dicts_list,train_blk_set,test_blocks_set,class_2,clean_flag,sliding_time_tr,sliding_time_te,window_time_length,window_type,channels_to_remove,number_of_channels,p,p_num,train_blk_name,test_blk_name,path):

#     for class_1 in class_1_list:
#         import time
#         start_time = time.time()
#         X_tr, Y_tr = trials_set_builder(data_dicts_list[p],train_blk_set,'TRAIN',class_1,class_2,clean_flag,sliding_time_tr,window_time_length,window_type,channels_to_remove,number_of_channels)
#         X_te, Y_te = trials_set_builder(data_dicts_list[p],test_blocks_set,'TEST',class_1,class_2,clean_flag,sliding_time_te,window_time_length,window_type,channels_to_remove,number_of_channels)

#         print(X_tr.shape,Y_tr.shape,"train shape")
#         print(X_te.shape,Y_te.shape,"test shape")

#         [train_features, test_features] = feature_extractor(X_tr, Y_tr, number_of_bands, X_te,number_of_components)
#         print(train_features.shape, "train_features shape")
#         print(test_features.shape, "test_features shape")
#         selected_features = feature_selector(train_features, Y_tr, number_of_selected_features)


#         clf = classifier
#         runs = 1
#         train_acc_list = []
#         test_acc_list = []
#         vote_acc_list = []

#         train_acc_list,test_acc_list,vote_acc_list = runs_iterattor(runs,clf,train_features[:,selected_features],Y_tr[:,0],test_features[:, selected_features],Y_te,test_acc_list,train_acc_list,vote_acc_list)
        

#         end_time = time.time()
#         running_time = end_time-start_time
#         participant = p_num
#         class1 = class_1
#         class2 = class_2
#         running_time = running_time
#         test_acc = np.average(test_acc_list)
#         train_acc = np.average(train_acc_list)
#         vote_acc = np.average(vote_acc_list)
#         test_size = X_te.shape
#         train_size = X_tr.shape
#         train_block = train_blk_name
#         test_block = test_blk_name
#         new_row = [participant, class1, class2,running_time,test_acc,train_acc,test_size,train_size,train_block,test_block,vote_acc]

#         save_csv(new_row,path)


#         print(train_acc_list,"train",class_1)
#         print(test_acc_list,"test",class_1)



In [63]:
def save_csv(new_row, path):

    absolute_path = os.path.abspath(path)
    try:
        rf = pd.read_csv(absolute_path)
    except FileNotFoundError:
        rf = pd.DataFrame(columns=column_names_v2)

    new_row_df = pd.DataFrame([new_row], columns=column_names_v2)
    cf = pd.concat([rf, new_row_df], ignore_index=True)
    cf.to_csv(absolute_path, index=False)

In [52]:
tr_pca,te_pca = apply_pca(X_tr,X_te,50)
print(tr_pca.shape,te_pca.shape)

(275, 50) (110, 50)


# Running Network

In [143]:
# PATH = '/home/mahdi146/projects/def-b09sdp/mahdi146/Cedar/Classification/EEG/Results/XGBoost/'
# class_1_list = ['Hand','Feet','Tongue','Mis']
# class_2 = 'Rest'
# p_num_list = [9]
# train_blocks_set = [0,1,2,3,4]
# test_blocks_set = [5,6]
# window_time_length = 4
# sliding_time_tr = 4
# sliding_time_te = 4
# vote_window = 4
# number_of_selected_features = 10
# # channels_to_remove = [list(range(1,56))]
# channels_to_remove = []
# number_of_channels = NUMBER_OF_CHANNELS
# params = {
#     'max_depth': 5,
#     'min_child_weight': 1,
#     'gamma': 0,
#     'subsample': 0.8,
#     'colsample_bytree': 0.8,
#     'learning_rate': 0.1,
# }




# classifier_dic = {"XGB":XGBClassifier(),"LDA":LDA(),"RF":RF()}
# number_of_components_list = [10,12,14,8]
# number_of_selected_features_list = [10,15,20]
# overlapflag = False
# window_time_length_list = [4,2,1]
# window_type_list = ["Rec","Kaiser","Hanning","Hamming"]
# train_blk_set_dic = {"01234":[0,1,2,3,4],"0123":[0,1,2,3],,"012":[0,1,2],,"01":[0,1],"0":[0]}

classifier_dic = {"XGB":XGBClassifier()}
number_of_components_list = [10]
number_of_selected_features_list = [10]
channels_to_remove = []
number_of_channels = NUMBER_OF_CHANNELS
overlapflag = "Without_Overlap"
window_time_length_list = [2,1]
sliding_time_tr = 4
sliding_time_te = 4
vote_window = 4
window_type_list = ["Rec","Kaiser","Hamming","Hanning"]
train_blk_set_dic = {"12345":[0,1,2,3,4],"1234":[0,1,2,3],"123":[0,1,2],"12":[0,1],"1":[0]}
test_blk_set_dic = {"67":[5,6],"567":[4,5,6],"4567":[3,4,5,6],"34567":[2,3,4,5,6],"234567":[1,2,3,4,5,6]}
# train_blk_set_dic = {"12345":[0,1,2,3,4]}
# test_blk_set_dic = {"67":[5,6]}
class_1_list = ['Hand','Feet','Tongue','Mis']
class_2 = 'Rest'
p_num_list = [3,4,5,6,7,9,10,11,13,14]
PATH = "/home/mahdi146/projects/def-b09sdp/mahdi146/Cedar/Classification/EEG/Results-v2/"


for classifier, classifier_name in zip(classifier_dic.values(),classifier_dic.keys()):
    for number_of_components in number_of_components_list:
        for number_of_selected_features in number_of_selected_features_list:
                for window_time_length in window_time_length_list:
                    if overlapflag == "With_Overlap":
                        window_type_list = ["Rec"]  
                        sliding_time_tr = sliding_time_te = window_time_length/2
                    else:
                        window_type_list = ["Rec","Kaiser","Hamming","Hanning"]
                        sliding_time_tr = sliding_time_te = window_time_length

                    for window_type in window_type_list:
                        for train_blk_set,train_blk_name,test_blk_set,test_blk_name in zip(train_blk_set_dic.values(),train_blk_set_dic.keys(),test_blk_set_dic.values(),test_blk_set_dic.keys()):
                            p = 0
                            for p_num in p_num_list:
                                for class_1 in class_1_list:
                                    import time
                                    start_time = time.time()
                                    X_tr, Y_tr = trials_set_builder(data_dicts_list[p],train_blk_set,'TRAIN',class_1,class_2,True,sliding_time_tr,window_time_length,window_type,channels_to_remove,number_of_channels)
                                    X_te, Y_te = trials_set_builder(data_dicts_list[p],test_blk_set,'TEST',class_1,class_2,True,sliding_time_te,window_time_length,window_type,channels_to_remove,number_of_channels)

                                    print(X_tr.shape,Y_tr.shape,"train shape")
                                    print(X_te.shape,Y_te.shape,"test shape")

                                    [train_features, test_features] = feature_extractor(X_tr, Y_tr, number_of_bands, X_te,number_of_components)
                                    # [train_features, test_features] =  apply_pca(X_tr,X_te,10)
                                    print(train_features.shape, "train_features shape")
                                    print(test_features.shape, "test_features shape")
                                    selected_features = feature_selector(train_features, Y_tr, number_of_selected_features)

                         
                                    clf = classifier
                                    runs = 1
                                    train_acc_list = []
                                    test_acc_list = []
                                    vote_acc_list = []

                                    for r in range(runs):
                                        clf.fit(train_features[:, selected_features], Y_tr[:,0])
                                        
                                        y_pr_te = clf.predict(test_features[:, selected_features])
                                        y_pr_tr = clf.predict(train_features[:,selected_features])

                                        accuracy_te = accuracy_score(Y_te, y_pr_te)
                                        test_acc_list.append(accuracy_te)


                                        accuracy_tr = accuracy_score(Y_tr,y_pr_tr)
                                        train_acc_list.append(accuracy_tr)

                       
                                        y_pr_te_Vote = majority_vote_sliding_with_prev_v2(y_pr_te,vote_window)
                                        Y_te_Vote = majority_vote_sliding_with_prev_v2(Y_te.reshape(-1),vote_window)
                                        vote_acc, num_of_mismatches ,mismatches_list = custom_accuracy(Y_te_Vote,y_pr_te_Vote)
                                        vote_acc_list.append(vote_acc)
                                        
 
                                    end_time = time.time()
                                    running_time = end_time-start_time
                                    participant = p_num
                                    class1 = class_1
                                    class2 = class_2
                                    running_time = running_time
                                    test_acc = np.average(test_acc_list)
                                    train_acc = np.average(train_acc_list)
                                    vote_acc = np.average(vote_acc_list)
                                    test_size = X_te.shape
                                    train_size = X_tr.shape
                                    train_block = train_blk_name
                                    test_block = test_blk_name
                                    new_row = [participant, class1, class2,running_time,test_acc,train_acc,test_size,train_size,train_block,test_block,vote_acc]

                                    if overlapflag == "With_Overlap":
                                        path = os.path.join(
                                            PATH,
                                            classifier_name,
                                            f"{number_of_components}_CSP_Components",
                                            f"{number_of_selected_features}-Selected_Features",
                                            overlapflag,
                                            f"{window_time_length}_window_time_length",
                                            "50% Overlap",
                                            f"{train_blk_name}_Train/"
                                        )
                                    else:
                                        path = os.path.join(
                                            PATH,
                                            classifier_name,
                                            f"{number_of_components}_CSP_Components",
                                            f"{number_of_selected_features}-Selected_Features",
                                            overlapflag,
                                            f"{window_time_length}_window_time_length",
                                            f"{window_type}_Window",
                                            f"{train_blk_name}_Train/"
                                        )

                                    save_csv(new_row, path + f"P{p_num}.csv")
                                    print(
                                    f"classifier_name = {classifier_name}\n"
                                    f"number_of_components = {number_of_components}\n"
                                    f"number_of_selected_features = {number_of_selected_features}\n"
                                    f"overlapflag = {overlapflag}\n"
                                    f"window_time_length = {window_time_length}\n"
                                    f"window_type = {window_type}\n"
                                    f"train_blk_name = {train_blk_name}\n"
                                    f"Participant = {p_num}"
                                    )

                                    print(train_acc_list,"train",class_1)
                                    print(test_acc_list,"test",class_1)
                                    
                                p+=1
                            get_results_average(path,p_num_list,class_1_list)








###########################################################################
                
# p = 0
# for p_num in p_num_list:
#     for class_1 in class_1_list:
#         import time
#         start_time = time.time()
#         X_tr, Y_tr = trials_set_builder(data_dicts_list[p],train_blocks_set,'TRAIN',class_1,class_2,True,sliding_time_tr,channels_to_remove,number_of_channels)
#         X_te, Y_te = trials_set_builder(data_dicts_list[p],test_blocks_set,'TEST',class_1,class_2,True,sliding_time_te,channels_to_remove,number_of_channels)

#         print(X_tr.shape,Y_tr.shape,"train shape")
#         print(X_te.shape,Y_te.shape,"test shape")

#         [train_features, test_features] = feature_extractor(X_tr, Y_tr, number_of_bands, X_te)
#         # [train_features, test_features] =  apply_pca(X_tr,X_te,10)
#         print(train_features.shape, "train_features shape")
#         print(test_features.shape, "test_features shape")
#         selected_features = feature_selector(train_features, Y_tr, number_of_selected_features)

#         train_acc_list = []
#         test_acc_list = []

#         clf = XGBClassifier()
#         for r in range(1):
#             clf.fit(train_features[:, selected_features], Y_tr[:,0])
            
#             y_pr_te = clf.predict(test_features[:, selected_features])
#             y_pr_tr = clf.predict(train_features[:,selected_features])


#             accuracy_te = accuracy_score(Y_te, y_pr_te)
#             test_acc_list.append(accuracy_te)

#             accuracy_tr = accuracy_score(Y_tr,y_pr_tr)
#             train_acc_list.append(accuracy_tr)

#             # clf.fit(train_features[:, :], Y_tr[:,0])

#             # y_pr_te = clf.predict(test_features[:, :])
#             # y_pr_tr = clf.predict(train_features[:,:])

#             # accuracy_te = accuracy_score(Y_te, y_pr_te)
#             # test_acc_list.append(accuracy_te)

#             # accuracy_tr = accuracy_score(Y_tr,y_pr_tr)
#             # train_acc_list.append(accuracy_tr)

            




#             # for i in range(len(Y_te)):
#             #     print(f"Test_Real: {Y_te[i][0]}   Test_Predication: {y_pr_te[i]}")
            
#             # count_0s = np.count_nonzero(Y_tr == 0)
#             # count_1s = np.count_nonzero(Y_tr == 1)

#             # print(f"Number of 0s: {count_0s}")
#             # print(f"Number of 1s: {count_1s}")

#             # print("selected_features: ",selected_features)
#             # print(Y_te.shape,y_pr_te.shape,"shape ")
#             y_pr_te_Vote = majority_vote_sliding_with_prev_v2(y_pr_te,vote_window)
#             Y_te_Vote = majority_vote_sliding_with_prev_v2(Y_te.reshape(-1),vote_window)

#             # for i in range(len(Y_te)):
#             #     print(f"Test_Real_Vote: {Y_te_Vote[i]}   Test_Predication_vote: {y_pr_te_Vote[i]}")


#             acc, num_of_mismatches ,mismatches_list = custom_accuracy(Y_te_Vote,y_pr_te_Vote)
#             # print(acc,num_of_mismatches,mismatches_list, "acc, num_of_mismatches ,mismatches_list",class_1)





#         end_time = time.time()
#         running_time = end_time-start_time
#         participant = p_num
#         class1 = class_1
#         class2 = class_2
#         running_time = running_time
#         test_acc = np.average(test_acc_list)
#         train_acc = np.average(train_acc_list)
#         test_size = X_te.shape
#         train_size = X_tr.shape
#         train_block = '01234'
#         test_block = '56'
#         test_acc_vote = acc





#         new_row = [participant, class1, class2,running_time,test_acc,train_acc,test_size,train_size,train_block,test_block,test_acc_vote]

#         new_row_df = pd.DataFrame([new_row], columns=column_names_v2)
#         rf = pd.read_csv(PATH +'P'+str(p_num)+'.csv')
#         cf = pd.concat([rf, new_row_df], ignore_index=True)
#         cf.to_csv(PATH +'P'+str(p_num)+'.csv',index=False)



#         print(train_acc_list,"train",class_1)
#         print(test_acc_list,"test",class_1)
        
#     p+=1

After concatenation - final_data shape: (56, 64, 1000) final_labels shape: (56, 1)
After concatenation - final_data shape: (84, 64, 1000) final_labels shape: (84, 1)
After concatenation - final_data shape: (112, 64, 1000) final_labels shape: (112, 1)
After concatenation - final_data shape: (140, 64, 1000) final_labels shape: (140, 1)
After concatenation - final_data shape: (56, 64, 1000) final_labels shape: (56, 1)
(140, 64, 1000) (140, 1) train shape
(56, 64, 1000) (56, 1) test shape
Frequency range:  4
Frequency range:  8
Frequency range:  12
Frequency range:  16
Frequency range:  20
Frequency range:  24
Frequency range:  28
Frequency range:  32
Frequency range:  36
(140, 90) train_features shape
(56, 90) test_features shape
classifier_name = XGB
number_of_components = 10
number_of_selected_features = 10
overlapflag = Without_Overlap
window_time_length = 4
window_type = Hamming
train_blk_name = 12345
Participant = 3
[1.0] train Hand
[0.8392857142857143] test Hand
After concatenation 

# Side Analysis

In [8]:
# #Frame Maker
PATH = '/home/mahdi146/projects/def-b09sdp/mahdi146/Cedar/Classification/EEG/Results-v2/XGB/10_CSP_Components/10-Selected_Features/With_Overlap/Rec_Window/50%_Overlap/12345_Train/'
df = pd.read_csv(PATH+'frame.csv')
p_num_list = [3,4,5,6,7,9,10,11,13,14]
for p_num in p_num_list:
    df.to_csv(PATH+'P'+str(p_num)+'.csv',index=False)



In [137]:
path = os.path.join(
    PATH,
    classifier_name,
    f"{number_of_components}_CSP_Components",
    f"{number_of_selected_features}-Selected_Features",
    overlapflag,
    f"{window_time_length}_window_time_length",
    f"{window_type}_Window",
    f"{train_blk_name}_Train/",
)
print(path + f"P{p_num}.csv")

/home/mahdi146/projects/def-b09sdp/mahdi146/Cedar/Classification/EEG/Results-v2/XGB/10_CSP_Components/10-Selected_Features/Without_Overlap/4_window_time_length/Rec_Window/12345_Train/P10.csv


In [61]:
import os
import pandas as pd

def save_csv(new_row, classifier, number_of_components, number_of_selected_features, overlapflag,
             window_time_length, window_type, train_blk_name, p_num):
    # Convert to relative path
    relative_path = os.path.join(
        classifier,
        f"{number_of_components}_CSP_Components",
        f"{number_of_selected_features}-Selected_Features",
        overlapflag,
        f"{window_time_length}_window_time_length",
        window_type,
        f"{train_blk_name}_Train",
        f"P{p_num}.csv"
    )

    # Convert relative path to absolute path
    absolute_path = os.path.abspath(relative_path)

    # Read existing CSV or create an empty DataFrame
    try:
        rf = pd.read_csv(absolute_path)
    except FileNotFoundError:
        rf = pd.DataFrame(columns=column_names_v2)

    # Create a new row DataFrame
    new_row_df = pd.DataFrame([new_row], columns=column_names_v2)

    # Concatenate existing DataFrame and new row DataFrame
    cf = pd.concat([rf, new_row_df], ignore_index=True)

    # Write the concatenated DataFrame to CSV
    cf.to_csv(absolute_path, index=False)

# Example usage
classifier = "XGBClassifier"
number_of_components = 10
number_of_selected_features = 10
overlapflag = "With_Overlap"
window_time_length = "4_window_time_length"
window_type = "50% Overlap"
train_blk_name = "12345"
p_num = 9

new_row = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 10]
save_csv(new_row, classifier, number_of_components, number_of_selected_features, overlapflag,
         window_time_length, window_type, train_blk_name, p_num)


OSError: Cannot save file into a non-existent directory: '/home/mahdi146/XGBClassifier/10_CSP_Components/10-Selected_Features/Without_Overlap/4_window_time_length_window_time_length/50% Overlap/12345_Train'

In [43]:

# # Specific path and file name
# file_path = '/home/mahdi146/projects/def-b09sdp/mahdi146/Cedar/Classification/EEG/Results/XGBoost/frame.csv'  # Replace with your specific path
# df = pd.DataFrame(columns=column_names)
# # Write the DataFrame to a CSV file at the specified path
# df.to_csv(file_path, index=False)

In [38]:
first_list = ["12345", "1234", "123", "12", "1"]

second_list = []

for item in first_list:
    result = ""
    for i in range(len(item) - 1):
        result += str(int(item[i]) + int(item[i + 1]))
    second_list.append(result)

print(second_list)


['3579', '357', '35', '3', '']


In [33]:
train_blk_set_dic = {"01234":[0,1,2,3,4],"0123":[0,1,2,3],"012":[0,1,2],"01":[0,1],"0":[0]}
for train_blk_set,train_blk_name in zip(train_blk_set_dic.values(),train_blk_set_dic.keys()):
    print(len(train_blk_set))
    print(train_blk_name)

5
01234
4
0123
3
012
2
01
1
0


In [48]:
def remove_outliers(df):
    # Specify the threshold for outliers (you can adjust this based on your data)
    threshold = 10 ** 5

    # Calculate median and MAD for each row
    median = df.iloc[:, :-1].median(axis=1)
    mad = np.median(np.abs(df.iloc[:, :-1].sub(median, axis=0)), axis=1)
    threshold_array = median + threshold * mad

    # Identify rows with values exceeding the threshold
    outliers = df.iloc[:, :-1].gt(threshold_array[:, None], axis=0).any(axis=1)

    # Remove rows identified as outliers
    clean_df = df[~outliers]

    return clean_df


In [41]:
from scipy import stats

def remove_outliers_across_channels(df, threshold):
    # print(df.shape,"shape0")

    data_columns = df.columns[:-1]  # Excluding the label column by default

    # Separate the label column
    labels = df.iloc[:, -1]  # Assuming the label is in the last column
    data_without_label = df.iloc[:, :-1]  # DataFrame without the label column

    # Calculate Z-scores for each row
    # print(data_without_label.shape,"shape1")
    # print(data_without_label[data_columns].shape,"shape2")
    # print(data_without_label.head(10))
    data_without_label[data_columns] = data_without_label[data_columns].apply(pd.to_numeric, errors='coerce')
    z_scores = stats.zscore(data_without_label[data_columns], axis=1)
    abs_z_scores = abs(z_scores)

    filtered_entries = (abs_z_scores < threshold).all(axis=1)
    filtered_data = data_without_label[filtered_entries]

    # Remove corresponding labels for removed rows
    filtered_labels = labels[filtered_entries]

    # If you want to reset index after filtering
    filtered_data.reset_index(drop=True, inplace=True)
    filtered_labels.reset_index(drop=True, inplace=True)

    # Combine the filtered data and labels
    filtered_df = pd.concat([filtered_data, filtered_labels], axis=1)

    return filtered_df


In [41]:
prediction_list = [1, 1, 1, 1, 0, 0, 1, 1, 0, 0]  # Replace with your actual prediction list

result = majority_vote_sliding_with_next(prediction_list)
result2 = majority_vote_sliding_with_prev_v2(prediction_list)
print("Majority Votes:", result)
print("Majority Votes Previous:", result2)

Majority Votes: [1, 1, 1, 0, 0, 1, 1, 0]
Majority Votes Previous: [1, 1, 1, 1, 1, 0, 0, 1, 1, 0]


## Getting Average

In [31]:
PATH = '/home/mahdi146/projects/def-b09sdp/mahdi146/Cedar/Classification/EEG/Results/XGBoost/Win4Shift2/'
p_num_list = [3,4,5,6,7,9,10,11,13,14]
vf = pd.DataFrame(columns=column_names_v2) 
for p_num in p_num_list:
    print(p_num)
    rf = pd.read_csv(PATH + "P" + str(p_num) + ".csv")
    vf = pd.concat([vf, rf], ignore_index=True)
vf.to_csv(PATH+ 'ResultsOfAll.csv', index=False)
# vf.tail()
    
columnNames = ['class','test_acc','vote_acc']
kf = pd.DataFrame(columns=columnNames)
kf.to_csv(PATH+'AverageAcc.csv',index=False)

vf = pd.read_csv(PATH +"ResultsOfAll.csv")
df = vf

class_list=['Hand','Feet','Tongue','Mis']
blk_list = [1234]
for class_ in class_list:
    avg_list = []
    avg_vote_list = []
    for blk in blk_list:
        gf = df[(df['train_block'] == blk) & (df['class1'] == class_)]
        avg = gf['test_acc'].mean()
        avg_vote = gf['test_acc_vote'].mean()
        avg_list.append(avg)
        avg_vote_list.append(avg_vote)
    print(avg_list)    
    new_row = [class_, avg_list[0],avg_vote_list[0]] 
    new_row_df = pd.DataFrame([new_row], columns=columnNames)
    rf = pd.read_csv(PATH + 'AverageAccExcP8.csv')
    cf = pd.concat([rf, new_row_df], ignore_index=True)
    cf.to_csv(PATH +'AverageAccExcP8.csv',index=False)  
kf = pd.read_csv(PATH +'AverageAccExcP8.csv') 
kf.head()

3
4
5
6
7
9
10
11
13
14
[0.7518181818181817]
[0.7381818181818182]
[0.7018181818181818]
[0.6981818181818181]


,class,b1234_test,b1234_test_vote
0,Hand,0.751818,0.770000
1,Feet,0.738182,0.740909
2,Tongue,0.701818,0.699091
3,Mis,0.698182,0.700000


In [139]:
def get_results_average(path,p_num_list,class_list):
    PATH = path
    vf = pd.DataFrame(columns=column_names_v2) 
    for p_num in p_num_list:
        print(p_num)
        rf = pd.read_csv(PATH + "P" + str(p_num) + ".csv")
        vf = pd.concat([vf, rf], ignore_index=True)
    vf.to_csv(PATH+ 'ResultsOfAll.csv', index=False)

    columnNames = ['class','test_acc','vote_acc']
    kf = pd.DataFrame(columns=columnNames)
    kf.to_csv(PATH+'AverageAcc.csv',index=False)
    vf = pd.read_csv(PATH +"ResultsOfAll.csv")
    df = vf
    blk_list = [1234]
    for class_ in class_list:
        gf = df[(df['class1'] == class_)]
        avg_test = gf['test_acc'].mean()
        avg_vote = gf['test_acc_vote'].mean() 
        new_row = [class_, avg_test,avg_vote] 
        new_row_df = pd.DataFrame([new_row], columns=columnNames)
        rf = pd.read_csv(PATH + 'AverageAcc.csv')
        cf = pd.concat([rf, new_row_df], ignore_index=True)
        cf.to_csv(PATH +'AverageAcc.csv',index=False)  
    kf = pd.read_csv(PATH +'AverageAcc.csv') 
    print(kf.head())


In [25]:
# Assuming df is your DataFrame with the last column named 'label'
data = {'col1': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
        'col2': ['some', 'random', 'data', 'for', 'example', 'purposes', 'in', 'this', 'case', 'it', 'does', 'not matter'],
        'label': ['a', 'a', 'a', 'b', 'b', 'b', 'a', 'a', 'a', 'b', 'b', 'b']}

df = pd.DataFrame(data)

print(df)
print("his")
# Define a custom sorting order based on the desired grouping
sorting_order = {'a': 0, 'b': 1}

# Create a new column with the sorting order
df['sorting_order'] = df.iloc[:, 2].map(sorting_order)

# Sort the DataFrame based on the new column and the original order within each group
df.sort_values(by=['sorting_order', df.columns[2]], inplace=True)

# Drop the temporary sorting column
df.drop('sorting_order', axis=1, inplace=True)

# Optional: Reset the index if needed
df.reset_index(drop=True, inplace=True)

# Display the sorted DataFrame
print(df)





    col1        col2 label
0      1        some     a
1      2      random     a
2      3        data     a
3      4         for     b
4      5     example     b
5      6    purposes     b
6      7          in     a
7      8        this     a
8      9        case     a
9     10          it     b
10    11        does     b
11    12  not matter     b
his
    col1        col2 label
0      1        some     a
1      2      random     a
2      3        data     a
3      7          in     a
4      8        this     a
5      9        case     a
6      4         for     b
7      5     example     b
8      6    purposes     b
9     10          it     b
10    11        does     b
11    12  not matter     b


In [54]:
data = {
    'col1': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,13],
    'col2': ['some', 'random', 'data', 'for', 'example', 'purposes', 'in', 'this', 'case', 'it', 'does', 'not matter','b'],
    'label': ['a', 'a', 'a', 'b', 'b', 'b', 'a', 'a', 'a', 'b', 'b', 'b','b']
}

df = pd.DataFrame(data)

x=0
i=0
class_1 = 'a'
class_2 = 'b'
sampleList = []
while i<len(df):
    if (df.iloc[i,2]==class_1):
        x+=1
    else:
        i-=1
        sampleList.append(x)
        x=0
        class_1,class_2 = class_2,class_1
    i+=1
sampleList.append(x)
print(sampleList)

[3, 3, 3, 4]


In [52]:
data = {
    'col1': [1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12],
    'col2': ['some', 'random', 'data', 'for', 'example', 'purposes', 'in', 'this', 'it', 'does', 'not matter'],
    'label': ['a', 'a', 'a', 'b', 'b', 'b', 'a', 'a', 'b', 'b', 'b']
}
df = pd.DataFrame(data)

print(get_group_start_indices(df))


[0, 3, 6, 8]


In [43]:
data = {
    'col1': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,13],
    'col2': ['some', 'random', 'data', 'for', 'example', 'purposes', 'in', 'this', 'case', 'it', 'does', 'not matter','c'],
    'label': ['a', 'a', 'a', 'b', 'b', 'b', 'a', 'a', 'a', 'b', 'b', 'b','b']
}

df = pd.DataFrame(data)

# Identify consecutive groups of 'a's by creating a new group ID each time 'label' changes from 'b' to 'a'
df['group'] = (df['label'] != df['label'].shift(1)).cumsum()

# Count occurrences of 'a' within each group
group_counts = df[df['label'] == 'a'].groupby('group').size()

group_counts_b = df[df['label'] == 'b'].groupby('group').size()
print(group_counts_b)
print(group_counts_b.index[0])
print(group_counts_b.iloc[0])
print(group_counts)

group
2    3
4    4
dtype: int64
2
3
group
1    3
3    3
dtype: int64


In [43]:
p_num = 8
b_num = 7
path = f'../../Participants/P{p_num}v5/'
mat = loadmat(path+'P'+str(p_num)+'B'+str(b_num)+'.mat', chars_as_strings=True, mat_dtype=True, squeeze_me=True, struct_as_record=False, verify_compressed_data_integrity=False, variable_names=None)
df = pd.DataFrame(mat['Data'])

path = f'../../Participants/P{p_num}v0/'
mat = loadmat(path+'P'+str(p_num)+'B'+str(b_num)+'.mat', chars_as_strings=True, mat_dtype=True, squeeze_me=True, struct_as_record=False, verify_compressed_data_integrity=False, variable_names=None)
df_1 = pd.DataFrame(mat['Data'])

print(df.shape)
print(df_1.shape)


FileNotFoundError: [Errno 2] No such file or directory: '../../Participants/P8v5/P8B7.mat'

In [57]:
from skimage.filters import median

def apply_median_filter(df, window_size=10):
    channel_data = df.iloc[:, :-1].values
    labels = df.iloc[:, -1].values 
    filtered_channel_data = np.zeros_like(channel_data) 

    for i in range(channel_data.shape[1]):
        print(i,"channel")
        filtered_channel_data[:, i] = median(channel_data[:, i], selem=np.ones(window_size))

    df_filtered = pd.DataFrame(filtered_channel_data, columns=df.columns[:-1])
    df_filtered['Label'] = labels
    print(df_filtered.shape, "shape after median filter")

    return df_filtered


(114281, 65)
(114281, 65)


In [59]:
extra_samples_block_counter(df_1,trial_order[0])

hi
[6191, 10157, 8157, 4065, 10161, 8156, 4060, 6014]
[8156, 8156, 6178, 10157, 10156, 4064, 4063, 6015]
[10158, 6176, 8165, 10155, 4073, 8156, 6184, 4016]
[10165, 10155, 6183, 6177, 4060, 4062, 8162, 8016]


In [82]:
for b in range(7):
    extra_samples_block_counter(data_dicts_list[-1][b],trial_order[b],b)

hi
Tongue
[3093, 5078, 4078, 2030, 5078, 4079, 2032, 3007]
Feet
[4078, 4078, 3087, 5079, 5078, 2030, 2035, 3007]
Mis
[5080, 3089, 4076, 5079, 2036, 4077, 3093, 2007]
Hand
[5080, 5078, 3092, 3088, 2035, 2030, 4077, 4007]
hi
Feet
[3091, 4082, 2034, 3093, 5082, 2034, 4079, 5007]
Mis
[4083, 2033, 5079, 5082, 2031, 3090, 3087, 4007]
Hand
[2033, 5078, 5079, 4078, 4079, 2036, 3090, 3007]
Tongue
[2033, 3089, 3092, 4078, 5082, 5082, 4079, 2008]
hi
Hand
[4078, 2035, 2036, 5082, 3089, 4083, 5083, 3008]
Feet
[5077, 3088, 4078, 2035, 3088, 5077, 2033, 4007]
Tongue
[3088, 4082, 5082, 3087, 2031, 5079, 4077, 2007]
Mis
[2037, 2035, 3093, 3091, 4076, 5079, 5081, 4007]
hi
Tongue
[3087, 5081, 4082, 2035, 5077, 4077, 2031, 3008]
Mis
[4082, 4083, 3089, 5156, 5078, 2111, 2026, 3007]
Hand
[5105, 3088, 4106, 5077, 2065, 4076, 3122, 2007]
Feet
[5077, 5116, 3090, 3121, 2026, 2049, 4079, 4008]
hi
Mis
[4077, 2034, 2061, 3092, 5171, 4082, 3165, 5008]
Feet
[3201, 4078, 4077, 5177, 2030, 3166, 5083, 2007]
Hand
[5079